In [2]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
options_df = pd.read_csv('Holzland_Options_URLs.csv', index_col=[0])

## Scraping page

In [65]:
browser.quit()

In [66]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)
url = 'https://www.holzland.de/p/konstruktionsholz-kieferfichte-kdi-gruen-gehobelt-gefast-staerke-ab-40-mm/HL1471671/?pmzr=10089&itemId=7001382347'
browser.get(url)

### Option three dims

In [67]:
sleep_time=0.2

In [75]:
%%time
three_dims_df = pd.DataFrame()
for p, idx in enumerate(options_df[options_df['Options']=='LÄNGE, BREITE, STÄRKE'].index[:]):
    url = options_df['URL'].loc[idx]
    browser.get(url)
    
    # Metadata for choosing Options
    path = '//*[@name="var[import:Laenge_Artikel]"]//option'
    l_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Breite_Artikel]"]//option'
    w_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Staerke_Artikel]"]//option'
    h_options = browser.find_elements_by_xpath(path)
    
    l,w,h = len(l_options), len(w_options), len(h_options)
    total = l*w*h
    
    print('Product', p, '|', "Combs:", total)
    
    product_df = pd.DataFrame()
    for i in range(total):
        l_idx = i//(total/l)+1
        w_idx = i%w+1
        h_idx = i%h+1

        # Length
        if l>0:
            path = '/html/body/div[2]/div[3]/div/div/div/div[3]/div/div[1]/div[1]/form/div/div[1]/select/option[{}]'.format(l_idx)
            exp_l =  browser.find_elements_by_xpath(path)[0].text
            exp_l = re.sub(r'[^\.\d]','', exp_l)
            browser.find_elements_by_xpath(path)[0].click()
            sleep(sleep_time)

        # Width
        if w>0:
            path = '/html/body/div[2]/div[3]/div/div/div/div[3]/div/div[1]/div[2]/form/div/div[1]/select/option[{}]'.format(w_idx)
            exp_w =  browser.find_elements_by_xpath(path)[0].text
            exp_w = re.sub(r'[^\.\d]','', exp_w)
            browser.find_elements_by_xpath(path)[0].click()
            sleep(sleep_time)

        # Height
        if h>0:
            path = '/html/body/div[2]/div[3]/div/div/div/div[3]/div/div[1]/div[3]/form/div/div[1]/select/option[{}]'.format(h_idx)
            exp_h =  browser.find_elements_by_xpath(path)[0].text
            exp_h = re.sub(r'[^\.\d]','', exp_h)
            browser.find_elements_by_xpath(path)[0].click()
                                                
        sleep(0.7)
        # Getting Main Data
        path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dd'
        values = [el.text for el in browser.find_elements_by_xpath(path)]

        path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dt'
        cols = [el.text for el in browser.find_elements_by_xpath(path)]
        main_data = pd.DataFrame([values], columns=cols)

        # Getting Dimentions
        path = '//*[@class="prd-details text--small"]'
        raw_dims = browser.find_elements_by_xpath(path)[0].text
        
        # Title
        path = '//*[@class="h3 headline"]'
        title = browser.find_elements_by_xpath(path)[0].text

        # Price
        path = '//*[@class="delivery-price-with-unit"]'
        price = browser.find_elements_by_xpath(path)[0].text

        #SKU
        path = '//*[@class="article-number text--small color-gray-light"]'
        sku = browser.find_elements_by_xpath(path)[0].text

        item_df = pd.DataFrame({'SKU':sku,
                                'Title':title,
                                'Raw Dims':raw_dims,
                                'Price':price,
                                'URL':browser.current_url}, index=[0])
                                            
        item_df = pd.concat([item_df, main_data], axis=1)
        
        product_df = pd.concat([product_df, item_df])
        sleep(0.3)
     
    three_dims_df = pd.concat([three_dims_df, product_df], ignore_index=True)
    #three_dims_df = three_dims_df[~three_dims_df.duplicated('SKU')]  
    #three_dims_df.to_csv('Holzland_3D_Options.csv')
    
    print('Product size:', product_df.shape[0], '|', 'All products size:', three_dims_df.shape[0])

Product 0 | Combs: 60
Product size: 60 | All products size: 60
Product 1 | Combs: 28
Product size: 28 | All products size: 88
Product 2 | Combs: 270
Product size: 270 | All products size: 358
Product 3 | Combs: 140
Product size: 140 | All products size: 498
Product 4 | Combs: 36
Product size: 36 | All products size: 534
Product 5 | Combs: 12
Product size: 12 | All products size: 546
Product 6 | Combs: 28
Product size: 28 | All products size: 574
Product 7 | Combs: 240
Product size: 240 | All products size: 814
Product 8 | Combs: 32
Product size: 32 | All products size: 846
Product 9 | Combs: 18
Product size: 18 | All products size: 864
Product 10 | Combs: 8
Product size: 8 | All products size: 872
Product 11 | Combs: 200
Product size: 200 | All products size: 1072
Product 12 | Combs: 24
Product size: 24 | All products size: 1096
Product 13 | Combs: 36
Product size: 36 | All products size: 1132
Product 14 | Combs: 28
Product size: 28 | All products size: 1160
Product 15 | Combs: 126
Pro

In [76]:
three_dims_df['Length'] = three_dims_df['Raw Dims'].str.split(' x ', expand=True)[0]
three_dims_df['Width'] = three_dims_df['Raw Dims'].str.split(' x ', expand=True)[1]
three_dims_df['Thickness'] = three_dims_df['Raw Dims'].str.split(' x ', expand=True)[2]
#three_dims_df = three_dims_df[~three_dims_df.duplicated('SKU')]
#three_dims_df[~three_dims_df.duplicated('SKU')].sort_values(['Title','Length','Width','Thickness'])#.tail(5)

In [130]:
test.isna().sum()/test.shape[0]*100

SKU                                 0.000000
Title                               0.000000
Raw Dims                            0.000000
Price                               0.000000
URL                                 0.000000
Verbindung                         98.615598
Nutzungsklasse                     98.615598
Länge                               0.000000
Breite                              0.000000
Stärke                              0.000000
Holzart / Dekornachbildung          3.461006
Verleimung                         98.200277
Furnierqualität                    86.340563
Furnierrichtung                    99.169359
Mittellage                         99.584679
Farbe                              99.353946
Marke                              86.340563
Dekornummer                        99.123212
Dekorbezeichnung                   99.123212
Dekorstruktur                      99.123212
Sortierklasse                      64.882326
Schnittklasse                      92.108906
Sortierkla

In [128]:
test[(~test.duplicated('SKU', keep='first')) & 
     (~test['URL'].str.contains('pmzr'))
    ].shape

(717, 38)

In [98]:
print(three_dims_df['URL'].loc[3657])

https://www.holzland.de/p/glattkantbrett-nordische-fichte-us-hobelfallend/HL496752/?pmzr=10110&itemId=7000537752


In [97]:
print(three_dims_df['URL'].loc[1992])

https://www.holzland.de/p/glattkantbrett-nordische-fichte-us-hobelfallend/HL496752/?itemId=7000537752
